<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/126_Docker_Container_Creation_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

👌 — let’s build you a **reusable Docker template** that you can drop into any ML or data science project. You’ve already got a good `.dockerignore`, so now let’s create a **solid Dockerfile template** that balances flexibility, reproducibility, and performance.

---

# 🧩 Template Dockerfile for ML/Data Projects

```dockerfile
# ----------------------
# 1. Base Image
# ----------------------
FROM python:3.11-slim

# Prevents Python from writing .pyc files and buffering stdout/stderr
ENV PYTHONDONTWRITEBYTECODE=1
ENV PYTHONUNBUFFERED=1

# ----------------------
# 2. System Dependencies
# ----------------------
RUN apt-get update && apt-get install -y --no-install-recommends \
    build-essential \
    curl \
    git \
    wget \
    ca-certificates \
    && rm -rf /var/lib/apt/lists/*

# ----------------------
# 3. Working Directory
# ----------------------
WORKDIR /app

# ----------------------
# 4. Install Python Dependencies
# ----------------------
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# ----------------------
# 5. Copy Project Code
# ----------------------
COPY . .

# ----------------------
# 6. Default Command
# ----------------------
CMD ["python", "main.py"]
```

---

# 📂 Project Structure Example

Here’s how your project folder might look with this setup:

```
my-project/
├── .dockerignore
├── Dockerfile
├── requirements.txt
├── main.py
├── src/
│   ├── data_preprocessing.py
│   ├── train.py
│   └── inference.py
└── notebooks/
```

---

# 📦 Example requirements.txt

Pin versions for reproducibility:

```
numpy==1.26.4
pandas==2.2.1
scikit-learn==1.5.0
matplotlib==3.9.1
jupyterlab==4.2.3
```

---

# ⚡ How to Use This Template

1. **Build the image**

   ```bash
   docker build -t myproject:0.1 .
   ```

2. **Run a container**

   ```bash
   docker run --rm -it myproject:0.1
   ```

3. **Run with Jupyter (interactive)**

   ```bash
   docker run --rm -it -p 8888:8888 myproject:0.1 jupyter lab --ip=0.0.0.0 --allow-root
   ```

   Then open `http://localhost:8888` in your browser.

4. **Mount local code/data (so you don’t rebuild every change)**

   ```bash
   docker run --rm -it -v $PWD:/app myproject:0.1 bash
   ```

---

# 🔑 Why This Template Works

* **Reproducible** → pinned Python + package versions.
* **Flexible** → system deps (compilers, curl, git) included for installing ML libs.
* **Clean** → `.dockerignore` prevents junk/secrets from bloating the image.
* **Ready for ML** → works for training, inference, or Jupyter workflows.

---

✅ With `.dockerignore` + this Dockerfile, you now have a **solid starting template** for any project.



## Docker Ignore File

In [ ]:
# Python cache
__pycache__/
*.pyc
*.pyo
*.pyd

# Virtual environments
venv/
env/
.venv/

# Jupyter Notebook checkpoints
.ipynb_checkpoints/

# OS files
.DS_Store
Thumbs.db

# Git and version control
.git
.gitignore
.gitattributes

# Logs & debug
*.log
*.out
*.err

# Data & models (better to mount these at runtime!)
data/
datasets/
*.csv
*.tsv
*.parquet
*.h5
*.hdf5
*.pth
*.pt
*.ckpt
*.joblib
*.pkl

# Large results
outputs/
results/
checkpoints/

# Environment & config (secrets should not be baked in!)
.env
*.secret
*.key


Let’s go slow and make this super practical.We’ll start with **Step 1: create two requirements files** (runtime vs. dev).
Do this on your **Mac host** in your project folder (not inside a running container).

# Step 1 — Create `requirements.txt` and `requirements-dev.txt`

### Create the files (copy–paste one of these options)

**Option A: use your editor (VS Code / nano)**

```bash
nano requirements.txt
```

Paste:

```
numpy==1.26.4
pandas==2.2.2
scikit-learn==1.5.1
```

Save/exit, then:

```bash
nano requirements-dev.txt
```

Paste:

```
jupyterlab==4.3.0
ipykernel==6.29.5
pytest==8.3.3
flake8==7.1.1
black==24.8.0
```

**Option B: one-liner heredocs**

```bash
cat > requirements.txt <<'EOF'
numpy==1.26.4
pandas==2.2.2
scikit-learn==1.5.1
EOF

cat > requirements-dev.txt <<'EOF'
jupyterlab==4.3.0
ipykernel==6.29.5
pytest==8.3.3
flake8==7.1.1
black==24.8.0
EOF
```

### Verify

```bash
ls -1
cat requirements.txt
cat requirements-dev.txt
```

### Why this split?

* `requirements.txt` = **runtime essentials** your app actually needs in production.
* `requirements-dev.txt` = **developer tooling** (Jupyter, tests, linters/formatters).
  Keeping these out of the prod image makes it **smaller, faster, and safer**.




### 🔑 Rename File

```bash
mv oldname.txt newname.txt
```

That tells the shell: *“move this file to the same location but with a new name.”*

## 📂 Renaming into another folder

You can also move/rename at the same time:

```bash
mv requirements.txt configs/requirements.txt
```




### 👀 View the Entire File

```bash
cat filename.txt
```

* Prints the whole file to the screen.
* Great for small files.

---

## 📖 Scroll Through a File

```bash
less filename.txt
```

* Opens the file in a pager so you can scroll.
* Navigation:

  * Space = page down
  * ↑/↓ arrows = scroll line by line
  * `q` = quit

---

## 🔝 View the Beginning

```bash
head filename.txt
head -n 20 filename.txt   # first 20 lines
```

---

## 🔚 View the End

```bash
tail filename.txt
tail -n 50 filename.txt   # last 50 lines
```



Instead of jumping into the Dockerfile, let’s first **understand the requirements files** you just created.

---

## 1. Runtime dependencies (`requirements.txt`)

These are the libraries your project *needs to run*. Think of them as the **core ingredients**.

* **numpy==1.26.4**

  * The foundation of scientific computing in Python.
  * Provides fast arrays, matrices, math functions.
  * Almost every data/ML library depends on it.

* **pandas==2.2.2**

  * Built on top of NumPy.
  * Provides `DataFrame` and `Series` for working with tabular data (like spreadsheets or SQL tables).
  * Super common in data wrangling, ETL, feature engineering.

* **scikit-learn==1.5.1**

  * Classic machine learning toolkit.
  * Algorithms: regression, classification, clustering, feature selection, pipelines.
  * Great for small-to-medium ML projects.

👉 Together, these 3 = the **“core data science stack”**.

---

## 2. Development dependencies (`requirements-dev.txt`)

These are tools you use while **developing**, but they aren’t strictly needed to *run* the project in production. Think of them as the **kitchen tools** vs. the ingredients.

* **jupyterlab==4.3.0**

  * Modern interface for Jupyter notebooks.
  * Lets you experiment interactively.

* **ipykernel==6.29.5**

  * Needed by Jupyter to run Python code inside notebooks.
  * Think of it as the “engine” Jupyter uses.

* **pytest==8.3.3**

  * Testing framework.
  * Lets you write unit tests to make sure your code works as expected.

* **flake8==7.1.1**

  * Linter (code style checker).
  * Helps catch errors like unused imports, undefined variables.

* **black==24.8.0**

  * Code formatter.
  * Automatically formats Python code in a clean, consistent style.

👉 These make your **developer experience smoother and safer**.

---

## 3. Why these versions?

* I chose the **latest stable releases** at the time of writing.
* Pinning exact versions (`==`) ensures **reproducibility**:

  * If you or someone else builds this project 6 months later, you’ll both get the *same versions* → fewer “works on my machine” issues.

How to pick versions in general:

* Start with latest stable release.
* Pin it (`==`) so you know what you’re using.
* If something breaks later, you can upgrade carefully.

---

## 4. Analogy

* `requirements.txt` = ingredients for the dish (must-have to cook).
* `requirements-dev.txt` = chef’s knives and measuring cups (makes cooking easier, but you don’t serve them to the customer).

---

✅ So right now, your setup is **very clean**:

* You have a minimal, production-ready runtime environment.
* You have an optional dev environment with Jupyter and testing tools.





## ✅ Why we use `requirements.txt` (and pin versions)

* **Reproducibility** → You (or a teammate, or future you) can recreate the *exact same environment* anywhere: laptop, server, Docker, cloud.
* **Stability** → By pinning versions (`numpy==1.26.4`), you freeze things in place so that a new release won’t silently break your code.
* **Debugging made easier** → If someone reports a bug, you can say “I was using Pandas 2.2.2” instead of “whatever version happened to be latest at the time.”
* **Controlled updates** → Instead of being surprised by an update, you can upgrade intentionally when you’re ready, test things, and then bump the version number in the file.

---

## 🧪 Example Scenario

* Today: Your project works with `pandas==2.2.2`.
* Next month: Pandas 2.3.0 releases, but it changes some behavior that breaks your data pipeline.
* Without version pinning → your `pip install pandas` would grab 2.3.0 automatically and suddenly your code doesn’t work.
* With version pinning (`pandas==2.2.2`) → your environment stays the same, safe and reproducible.

---

## 📦 In Docker

When Docker builds your image, it uses `requirements.txt` to **lock in the environment**.
So if you share your Dockerfile + requirements with someone else, they’ll get the **same working environment**, even if they’re on a different OS or machine.

---

✅ So yes: you’re protecting yourself from updates that could break code, and giving yourself a way to **rebuild exactly what worked before** anytime. That’s professional best practice.



this is the **Dockerfile** we’re talking about, and it’s a bit different from the `requirements.txt`. Let’s go line by line so it really makes sense.

---

## 🔑 First: How it’s different from `requirements.txt`

* `requirements.txt` = **Python-only dependencies** (numpy, pandas, sklearn).
* `Dockerfile` = **the full recipe for an environment**:

  * Which operating system (Ubuntu/Debian/Alpine, etc.)
  * Which Python version
  * System tools (git, curl, compilers, etc.)
  * Your Python requirements
  * Your project code
  * What command runs when the container starts

So:

* `requirements.txt` = *what ingredients you need to cook*.
* `Dockerfile` = *the entire kitchen, stove, and the recipe to cook it all*.

---

## 📄 Let’s break down your Dockerfile

```dockerfile
# ----------------------
# Base Image
# ----------------------
FROM python:3.11-slim
```

👉 Start from a lightweight Linux image that already has Python 3.11 installed.
“slim” = smaller size (fewer extra tools installed).

---

```dockerfile
# Set environment settings
ENV PYTHONDONTWRITEBYTECODE=1
ENV PYTHONUNBUFFERED=1
```

👉 These tweak Python’s behavior:

* `PYTHONDONTWRITEBYTECODE=1` → don’t generate `.pyc` cache files.
* `PYTHONUNBUFFERED=1` → force Python to flush output immediately (important for logging inside containers).

---

```dockerfile
# Install system dependencies
RUN apt-get update && apt-get install -y --no-install-recommends \
    build-essential \
    curl \
    git \
    wget \
    ca-certificates \
    && rm -rf /var/lib/apt/lists/*
```

👉 Install OS-level packages that many Python libs depend on:

* `build-essential` = compilers (for C/C++ extensions in NumPy, SciPy, etc.).
* `curl`, `wget` = download stuff from the internet.
* `git` = clone repos.
* `ca-certificates` = SSL support.
* `rm -rf /var/lib/apt/lists/*` → cleanup to keep image smaller.

---

```dockerfile
# Set workdir
WORKDIR /app
```

👉 Sets the default working directory inside the container.
Now everything you copy or run will happen inside `/app`.

---

```dockerfile
# Copy dependency files first (better layer caching)
COPY requirements.txt requirements-dev.txt ./
```

👉 Copy in your requirements files.
This is smart: if only your code changes, Docker will reuse cached layers and not reinstall dependencies every time.

---

```dockerfile
# Install runtime requirements
RUN pip install --no-cache-dir -r requirements.txt
```

👉 Install Python runtime deps (numpy, pandas, etc.).

---

```dockerfile
# If you want dev tools, build with:
# docker build -t myproject:dev --build-arg INSTALL_DEV=true .
ARG INSTALL_DEV=false
RUN if [ "$INSTALL_DEV" = "true" ] ; then pip install --no-cache-dir -r requirements-dev.txt ; fi
```

👉 This adds a “switch”:

* By default, dev tools (Jupyter, pytest, etc.) are NOT installed.
* But if you want them, you build with `--build-arg INSTALL_DEV=true`.

This keeps production images smaller, but lets you opt-in to dev tools when needed.

---

```dockerfile
# Copy the rest of your code
COPY . .
```

👉 Now copy in the actual project files (`main.py`, `src/`, etc.).

---

```dockerfile
# Default command
CMD ["python", "main.py"]
```

👉 The command that runs by default when the container starts.
You can override this when running the container (e.g. run Jupyter instead).

---

## 📌 Where does this file live?

Yes, you **save this as a file called `Dockerfile`** (no extension) in the **root of your project folder**, alongside:

```
myproject/
├── Dockerfile
├── .dockerignore
├── requirements.txt
├── requirements-dev.txt
├── main.py
└── src/
```

---

## 🛠️ How do you use it?

1. Build an image:

   ```bash
   docker build -t myproject:0.1 .
   ```
2. Run a container:

   ```bash
   docker run --rm myproject:0.1
   ```

   → Runs `python main.py` inside the container.

If you want dev tools:

```bash
docker build -t myproject:dev --build-arg INSTALL_DEV=true .
docker run -it -p 8888:8888 myproject:dev jupyter lab --ip=0.0.0.0 --allow-root
```

---

✅ So: the Dockerfile is your **full environment recipe**, while the requirements files are just the **Python dependency list**.




In [ ]:
# ----------------------
# Base Image
# ----------------------
FROM python:3.11-slim

# Set environment settings
ENV PYTHONDONTWRITEBYTECODE=1
ENV PYTHONUNBUFFERED=1

# Install system dependencies
RUN apt-get update && apt-get install -y --no-install-recommends \
    build-essential \
    curl \
    git \
    wget \
    ca-certificates \
    && rm -rf /var/lib/apt/lists/*

# Set workdir
WORKDIR /app

# Copy dependency files first (better layer caching)
COPY requirements.txt requirements-dev.txt ./

# Install runtime requirements
RUN pip install --no-cache-dir -r requirements.txt

# If you want dev tools, build with:
# docker build -t myproject:dev --build-arg INSTALL_DEV=true .
ARG INSTALL_DEV=false
RUN if [ "$INSTALL_DEV" = "true" ] ; then pip install --no-cache-dir -r requirements-dev.txt ; fi

# Copy the rest of your code
COPY . .

# Default command
CMD ["python", "main.py"]


🎉 You’ve got the idea. Let’s do this step by step so you also practice the terminal.

---

## 🛠️ Step 1: Create the project folder

On your **Mac terminal** (not inside a container):

```bash
mkdir myproject
cd myproject
```

---

## 🛠️ Step 2: Create the Dockerfile

Open it with nano:

```bash
nano Dockerfile
```

Paste this starter template:

```dockerfile
# ----------------------
# Base Image
# ----------------------
FROM python:3.11-slim

# Set environment settings
ENV PYTHONDONTWRITEBYTECODE=1
ENV PYTHONUNBUFFERED=1

# Install system dependencies
RUN apt-get update && apt-get install -y --no-install-recommends \
    build-essential \
    curl \
    git \
    wget \
    ca-certificates \
    && rm -rf /var/lib/apt/lists/*

# Set workdir
WORKDIR /app

# Copy dependency files first (better layer caching)
COPY requirements.txt requirements-dev.txt ./

# Install runtime requirements
RUN pip install --no-cache-dir -r requirements.txt

# If you want dev tools, build with:
# docker build -t myproject:dev --build-arg INSTALL_DEV=true .
ARG INSTALL_DEV=false
RUN if [ "$INSTALL_DEV" = "true" ] ; then pip install --no-cache-dir -r requirements-dev.txt ; fi

# Copy the rest of your code
COPY . .

# Default command
CMD ["python", "main.py"]
```

Save and exit:

* Press **Ctrl+O** (write out), then **Enter**.
* Press **Ctrl+X** (exit).

---

## 🛠️ Step 3: Verify

Check that it’s there:

```bash
ls -1
```

You should see:

```
Dockerfile
```





Right now you’re in the `DOCKER/` folder, and you’ve got both the `Dockerfile` and a `myproject/` folder sitting side by side. You want the `Dockerfile`, `requirements.txt`, and `requirements-dev.txt` to live **inside** `myproject/`.

---

## 🛠️ Step 1: Move files into `myproject`

Use `mv` for each file:

```bash
mv Dockerfile requirements.txt requirements-dev.txt myproject/
```

Now check:

```bash
ls myproject
```

You should see:

```
Dockerfile
requirements.txt
requirements-dev.txt
```

---

## 🛠️ Step 2: Confirm structure

```bash
ls -R myproject
```

This will recursively list everything inside `myproject`.

Expected right now:

```
myproject:
Dockerfile
requirements.txt
requirements-dev.txt
```




 🙌 your folder is now set up with the `Dockerfile` and requirements files in the right place. That’s a great checkpoint.

---

## 🛠️ Next Step: Add `.dockerignore`

This file will keep your image clean by excluding unnecessary stuff (caches, data, secrets, etc.).

From inside your `myproject/` folder, run:

```bash
nano .dockerignore
```

Paste this content:

```
# Python cache
__pycache__/
*.pyc
*.pyo
*.pyd

# Virtual environments
venv/
env/
.venv/

# Jupyter Notebook checkpoints
.ipynb_checkpoints/

# OS files
.DS_Store
Thumbs.db

# Git and version control
.git
.gitignore
.gitattributes

# Logs & debug
*.log
*.out
*.err

# Data & models (better to mount these at runtime!)
data/
datasets/
*.csv
*.tsv
*.parquet
*.h5
*.hdf5
*.pth
*.pt
*.ckpt
*.joblib
*.pkl

# Large results
outputs/
results/
checkpoints/

# Environment & config (secrets should not be baked in!)
.env
*.secret
*.key
```

Save (**Ctrl+O**, Enter) and exit (**Ctrl+X**).

---

## ✅ Verify

Check the file is there:

```bash
ls -a
```

You should now see:

```
.  ..  .dockerignore  Dockerfile  requirements.txt  requirements-dev.txt
```




`.dockerignore` is a **hidden file**, so it only shows up with `ls -a`. That’s totally normal and exactly how it should look.

Now you’ve got these files inside `myproject/`:

```
.dockerignore
Dockerfile
requirements.txt
requirements-dev.txt
```

---

## 🛠️ Next Step: Add `main.py`

This will be the “entrypoint” for your container. Let’s make a simple Python script that proves everything is working:

1. Open it with nano:

```bash
nano main.py
```

2. Paste this:

```python
import sys, platform, os

def main():
    name = os.getenv("NAME", "world")
    print(f"Hello, {name} 👋")
    print("Python version:", platform.python_version())
    print("OS:", platform.platform())
    print("Args:", sys.argv[1:])

if __name__ == "__main__":
    main()
```

3. Save (**Ctrl+O**, Enter) and exit (**Ctrl+X**).

---

## ✅ Verify

Check that it’s there:

```bash
ls
```

You should now see:

```
Dockerfile
main.py
requirements.txt
requirements-dev.txt
```

(and `.dockerignore` if you use `ls -a`).





## 1. `.dockerignore`

* This file tells Docker what **not to copy into your Docker image** when you run `docker build`.
* If `.env` is in `.dockerignore`:

  * Your `.env` file will **stay on your laptop**.
  * It will **not** get baked into the Docker image.
  * That means if you push the image to Docker Hub, your secrets are safe.

---

## 2. Git (`.gitignore`)

* Docker and Git are separate.
* If you want to make sure your `.env` file **never gets committed to GitHub**, you also need it in **`.gitignore`**.

Example `.gitignore` snippet:

```
.env
*.secret
*.key
```

---

## 3. So the full story

* `.dockerignore` → protects your **images**.
* `.gitignore` → protects your **repos**.

For maximum safety:

* Add `.env` to **both** `.dockerignore` and `.gitignore`.
* Load environment variables at runtime instead of hardcoding them.

---

## 4. Example: using `.env` with Docker

Let’s say you have a file `.env` like:

```
OPENAI_API_KEY=sk-xxxx
```

Run the container with it:

```bash
docker run --rm --env-file .env myproject:0.1
```

Inside the container, your Python code can access it:

```python
import os
print(os.getenv("OPENAI_API_KEY"))
```

---

✅ So to answer directly:

* With `.env` in `.dockerignore`, the file will **not leak into images**.
* With `.env` in `.gitignore`, it will **not leak into GitHub**.



This is **the heart of secret management** when working with Docker. Let’s break it down step by step:

---

## 🚫 What NOT to Do

* ❌ Don’t hardcode API keys into your Python files (`main.py`, etc.).
* ❌ Don’t put them in `requirements.txt`, `Dockerfile`, or commit `.env` to GitHub.
* ❌ Don’t copy `.env` into the Docker image — once it’s baked in, anyone with the image can see it.

---

## ✅ The Right Way: Keep Secrets Outside the Image

Think of Docker images as **blueprints you share**. Anything inside is visible to others.
Secrets should stay **outside the image** and only get injected at runtime.

You have two main ways to do this:

---

### **Option 1: Environment Variables**

1. Create a local `.env` file (not committed, and ignored by `.gitignore` + `.dockerignore`):

   ```
   OPENAI_API_KEY=sk-xxxx
   ANTHROPIC_API_KEY=claude-xxxx
   ```

2. Run the container and load the file:

   ```bash
   docker run --rm --env-file .env myproject:0.1
   ```

3. Inside your Python code (`main.py`):

   ```python
   import os

   openai_key = os.getenv("OPENAI_API_KEY")
   claude_key = os.getenv("ANTHROPIC_API_KEY")

   print("OpenAI key loaded?", bool(openai_key))
   ```

👉 This way, your keys **stay on your machine**, not in the image.

---

### **Option 2: Pass Variables Inline**

If you don’t want a `.env` file, you can pass them directly:

```bash
docker run --rm \
  -e OPENAI_API_KEY=sk-xxxx \
  -e ANTHROPIC_API_KEY=claude-xxxx \
  myproject:0.1
```

---

## 🔒 Security Best Practices

* Add `.env` to both `.gitignore` and `.dockerignore`.
* Never push `.env` to GitHub or Docker Hub.
* Rotate keys if you ever suspect they were exposed.
* In teams: use a secret manager (AWS Secrets Manager, Vault, or Docker secrets in Swarm/Kubernetes).

---

## 🧭 Mental Model

* **Dockerfile + code + requirements** → public/shareable recipe.
* **.env + secrets** → private, stays with you (injected only when running).

---

✅ So in your case:

* Keep your API keys in `.env` locally.
* Add `.env` to `.gitignore` (and `.dockerignore`).
* Inject them at runtime with `--env-file .env`.
* Access them inside your code with `os.getenv(...)`.





## ✅ 1. Your setup looks great

Right now inside `myproject/` you’ve got:

```
.dockerignore
Dockerfile
requirements.txt
requirements-dev.txt
```

That’s **exactly what we want so far**. The only thing missing is your Python entrypoint: `main.py`.

---

## 🛠️ 2. Next step: Create `main.py`

From inside `myproject/`:

```bash
nano main.py
```

Paste this starter code:

```python
import sys, platform, os

def main():
    name = os.getenv("NAME", "world")
    print(f"Hello, {name} 👋")
    print("Python version:", platform.python_version())
    print("OS:", platform.platform())
    print("Args:", sys.argv[1:])

if __name__ == "__main__":
    main()
```

Save (**Ctrl+O**, Enter) and exit (**Ctrl+X**).

---

## 📂 After this step your project should look like:

```
myproject/
├── .dockerignore
├── Dockerfile
├── requirements.txt
├── requirements-dev.txt
└── main.py
```






## 🛠️ Next Step: Build Your First Image

From inside the `myproject/` folder, run:

```bash
docker build -t myproject:0.1 .
```

* `docker build` = command to build an image.
* `-t myproject:0.1` = tag the image with the name `myproject` and version `0.1`.
* `.` = build context (the current folder).

---

## 🧪 Then, run the container

Once the build finishes:

```bash
docker run --rm myproject:0.1
```

Expected output:

```
Hello, world 👋
Python version: 3.11.x
OS: Linux-...
Args: []
```

---

✅ That’s the big milestone: you’ll have **built and run your first Dockerized Python app**.



Boom 🚀 nice work — you’ve just built and run your first Docker container from scratch! Now let’s explore what’s inside so you can see it really is a “little computer” running your code.

---

## 🛠️ Step 1: Run it interactively

So far you’ve run:

```bash
docker run --rm myproject:0.1
```

which just ran `main.py` and exited.

If you want to **get a shell inside the container**, do:

```bash
docker run -it myproject:0.1 bash
```

* `-it` = interactive terminal.
* `bash` = start a Bash shell instead of running `main.py`.

Your prompt will change to something like:

```
root@123abc:/app#
```

You’re now *inside the container’s Linux environment*. 🎉

---

## 🛠️ Step 2: Inspect the filesystem

Inside the container, try:

```bash
ls
```

You should see:

```
Dockerfile
requirements.txt
requirements-dev.txt
main.py
```

(the files you copied in with `COPY . .`).

Check the Python version:

```bash
python --version
```

See what processes are running:

```bash
ps aux
```

---

## 🛠️ Step 3: Explore from the outside

Even without starting a shell, you can **peek into a container**:

1. Run a container in the background:

   ```bash
   docker run -d --name mytest myproject:0.1 sleep 300
   ```

   * `-d` = detached (runs in background).
   * `--name mytest` = give it a name so you don’t have to copy the ID.
   * `sleep 300` = keep it alive for 5 minutes so you can inspect it.

2. Check running containers:

   ```bash
   docker ps
   ```

3. Exec into it:

   ```bash
   docker exec -it mytest bash
   ```

---

## 🛠️ Step 4: Inspect image layers

See how your image was built:

```bash
docker history myproject:0.1
```

List all images you’ve built:

```bash
docker images
```

---

✅ So you’ve got two flavors of inspection:

* **Inside the container**: `docker run -it ... bash`
* **From outside**: `docker ps`, `docker exec`, `docker history`, `docker images`




Ah, nice catch 👀 — that’s actually a good lesson about Docker images.

You’re inside `python:3.11-slim`, which is a **“slim” base image**.
👉 Slim images don’t include extra tools like `ps`, `top`, or even `man` pages — to keep them small.

That’s why `ps` is missing.

---

## ✅ Option 1: Install it yourself

Inside the container (as root), you could add it:

```bash
apt-get update && apt-get install -y procps
```

* `procps` is the package that provides `ps`, `top`, etc.
* After installing, you can run:

  ```bash
  ps aux
  ```

---

## ✅ Option 2: Use what’s already there

Even without `ps`, you can explore processes with:

```bash
cat /proc/self/status
cat /proc/cpuinfo
cat /proc/meminfo
```

These give you insights into the container’s own process, CPU, and memory.

---

## ✅ Option 3: Inspect from the outside

From your **Mac terminal** (outside the container), you can run:

```bash
docker top <container_id>
```

That shows processes running inside the container.

---

So don’t worry — nothing’s broken. It’s just that slim images strip out “nice-to-have” tools.

